# Qiskitを使って量子プログラミングに挑戦！

こんにちは！Qiskit Advocateの秋田です。2023年ももう半分終わっちゃいますね〜。私事ですが、今年の上半期は就活やら研究やらで生産性のあることが全然できていません！そもそも生産性のあることって何やねん。

さて、本日のイベントの内容ですが、

> PythonベースのOpen Source の量子プログラミングソフトウェア開発キットであるQiskit を使って、量子プログラミングを学びます。量子コンピューター初心者の方向けの量子プログラミング入門編です。
> 今回は、基本の量子ゲートの実装からクラウド型量子コンピューターでの実験まで行います。

とあるように、

> 「量子プログラミング、なにそれ美味しいの？」

という方向けにPythonでコーディングを学ぼう！というものになっております。

前提として、簡単な線形代数の知識、そしてPythonの基本文法が身についていることを想定しています。また、過去のイベントの動画も参考までに紹介します。

"[単一システムの「古典情報」〜 Qiskitテキストブック量子情報の基礎編](https://www.youtube.com/watch?v=_3UGr_2bgqo)" (発表者 : 外林さん)

"[Qiskitテキストブック「単一システム：量子情報」](https://www.youtube.com/watch?v=MdqMybfGeys)" (発表者 : 大野さん)

# 今回使うライブラリのインポート

Qiskitは複雑なので()、どのモジュールを使うかはしっかり考えなければいけません。また、Qiskitはローカルのコンピュータでもプログラムできますが、Quantum Labは環境が常に最新の状態になっているのでこちらを推奨します。

In [ ]:
import qiskit
from qiskit import Aer, QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit.visualization import plot_histogram
from qiskit_ibm_provider import IBMProvider

# Qiskitのバージョン確認
qiskit.__qiskit_version__

# 量子回路と量子ゲート

量子コンピュータ(ゲート方式)で問題を解くときに欠かせないのが、量子回路の設計です。

## 量子回路

早速量子回路を作るインスタンスを用意しましょう！

In [ ]:
qc = QuantumCircuit(2)

```python
QuantumCircuit(2)
```
この引数には、量子回路を作る量子ビットの数を入れます。今回の場合 $2$ なので $2$ 量子ビットの回路を作るよ！ということになります。

では、実際に回路が用意されているのかを見てみましょう！

```python
qc.draw("mpl")
```

これはmatplotlibの形式で回路を図示してくれます。

In [ ]:
qc.draw("mpl")

量子回路の表示のスタイルは他に

- latex
- text

があり、さらに他の引数を使うことで画像のデザインも出来ちゃいます！(時間の都合上、今回は紹介までに、参考は[こちら](https://qiskit.org/documentation/locale/ja_JP/tutorials/circuits_advanced/03_advanced_circuit_visualization.html))

話が前後してしまいますが、一旦量子回路作成の方に戻りましょう。というのも、量子回路のインスタンスの作り方は1通りだけでなく、いくつかあるため人によりその書き方が異なります。次は、レジスタというものを先に用意しましょう！

In [ ]:
qr = QuantumRegister(2)
qc = QuantumCircuit(qr)

qc.draw("mpl")

レジスタを用意すると、より複雑な問題(例えばエラー訂正など)を扱う際に便利だったりします。今回はそれほど高度なことをするつもりは無いので安心してください！

## 量子ゲート

量子回路を構成するのが量子ゲートです。今回は主にパウリゲート($X$, $Y$, $Z$)とアダマールゲート($H$)、そして $2$ 量子ビットゲートである $CNOT$ ゲートをご紹介します。まずは各ゲートについて、その特徴を理解しましょう！

### $X$ ゲート

$X$ ゲートは

> $X = \begin{pmatrix} 0 & 1 \\ 1 & 0 \\ \end{pmatrix}$

で定義され、次の変換を行います。

> $\begin{matrix} |0\rangle \rightarrow |1\rangle \\ |1\rangle \rightarrow |0\rangle \end{matrix}$

### $Y$ ゲート

$Y$ ゲートは

> $Y = \begin{pmatrix} 0 & -i \\ i & 0 \\ \end{pmatrix}$

で定義され、次の変換を行います。

> $\begin{matrix} |0\rangle \rightarrow i|1\rangle \\ |1\rangle \rightarrow -i|0\rangle \end{matrix}$

### $Z$ ゲート

$Z$ ゲートは

> $Z = \begin{pmatrix} 1 & 0 \\ 0 & -1 \\ \end{pmatrix}$

で定義され、次の変換を行います。

> $\begin{matrix} |0\rangle \rightarrow |0\rangle \\ |1\rangle \rightarrow -|1\rangle \end{matrix}$

### $H$ ゲート

$H$ ゲートは

> $H = \frac{1}{\sqrt 2} \begin{pmatrix} 1 & 1 \\ 1 & -1 \\ \end{pmatrix}$

で定義され、次の変換を行います。

> $\begin{matrix} |0\rangle \rightarrow \frac{1}{\sqrt 2}(|0\rangle + |1\rangle) \\ |1\rangle \rightarrow \frac{1}{\sqrt 2}(|0\rangle - |1\rangle) \end{matrix}$

### $CNOT$ ゲート

$CNOT$ ゲートは $2$ 量子ビットのゲートなので

> $CNOT = \begin{pmatrix} 1 & 0 & 0 & 0 \\ 0 & 1 & 0 & 0 \\ 0 & 0 & 0 & 1 \\ 0 & 0 & 1 & 0 \\ \end{pmatrix}$

で定義され、次の変換を行います。

> $\begin{matrix} |00\rangle \rightarrow |00\rangle \\ |01\rangle \rightarrow |01\rangle \\ |10\rangle \rightarrow |11\rangle \\ |11\rangle \rightarrow |10\rangle \\ \end{matrix}$

制御量子ビット( $1$ 量子ビット目)が $0$ であればターゲット量子ビット( $2$ 量子ビット目)に何もせず、制御量子ビットが $1$ であればターゲット量子ビットを反転させます。

量子ゲートのかけ方はそこまで複雑では無いため、一気に見てみましょう！

In [ ]:
qc = QuantumCircuit(qr)

qc.x(0)
# qc.y(0)
# qc.z(0)
# qc.h(0)
# qc.cx(0, 1)

qc.draw("mpl")

$X$, $Y$, $Z$, $H$ ゲートに関しては、それぞれ引数は1個でゲートをかける量子ビットのインデックスを入れます。 $CNOT$ ゲートに関しては、 (制御量子ビット, ターゲット量子ビット) の順でインデックスを入れます。

# 量子もつれ(エンタングルメント)

量子もつれについてまず軽く触れておきましょう！量子もつれ、またはエンタングルメントとは

> 量子もつれ $:=$ テンソル積で表せない状態

と定義できます。初心者の方の中では何を言っているのかサッパリわからないということもあると思います。ここではざっくりとしたイメージを掴めるようにしましょう！

量子もつれを理解する上で欠かせないのが「重ね合わせ」です。古典コンピュータと量子コンピュータの一番大きな違いとして考えられるこの重ね合わせですが、有名な話として「シュレディンガーの猫」というものがありますね。古典コンピュータでは、 $0$, $1$ の決定的なビットが扱われるのに対し、量子コンピュータでは、 $0$, $1$ の他に、例えば $0$ が出る確率が $30\%$, $1$ が出る確率が $70\%$ という確率的なビットが扱えます。このように、 $0$ が出るかもしれないし $1$ が出るかもしれないという2つの状態が「重なり合っている」ものを「重ね合わせ状態」と言います。

イメージとしてはコイントスです。

![コイントス](https://3.bp.blogspot.com/-3T4f37N8XqE/Wc8f5l8bqWI/AAAAAAABHJE/jFjmJTCU-30s0hGA5XFBoAeEed5aA0buQCLcBGAs/s800/sports_coin_toss.png)

コインを投げて掌の上や地面に落としたとき、我々はずっとコインを見られるのでコインが止まった瞬間どちらが上を向いているかがわかります。しかし、コインが空中を舞っている間に両手で覆い隠してしまったらどうでしょう？両手の中のコインは必ず表裏のどちらかを上に向けているのに、そのどちらかはわかりません。これを確認するには両手を開いて「観測(測定)」するしかありません。これが、重ね合わさっているイメージになります。

コインが1つの場合、「表」か「裏」かの2種類が重ね合わせられます。では、コインが2つの場合(コイントスは独立に行う)はどうでしょう？仮に、2つのコインを $a$, $b$ としましょう。 $a$, $b$ 共に両手で覆ってしまえば、

> ($a$, $b$) $=$ (表, 表), (表, 裏), (裏, 表), (裏, 裏)

という4種類の重ね合わせができます。今度は、 $b$ のみ両手で覆い、 $a$ は地面に落としてしまいましょう。そのとき、 $a$ は裏が上を向いているとします。すると

> ($a$, $b$) $=$ (裏, 表), (裏, 裏)

という2種類の重ね合わせのみになりますね。しかし、この重ね合わせの組み合わせというものを古典的な方法でコントロールするのは難しいのです。例えば、

> ($a$, $b$) $=$ (表, 表), (裏, 裏)

という重ね合わせの組を考えてみましょう。これを言い換えると、「 $a$ が表であれば $b$ も表、 $a$ が裏であれば $b$ も裏」という状況です。2つのコインどちらも、表裏どちらを取る可能性もあるので両方とも覆い隠す必要があります。ですがそうすると最初の結果に一致するはずです。そもそも2つのコイントスは独立であり、片方が分かればもう片方もわかるという状況があまりに不自然であるというのは理解が難しくないと思います。しかし、量子コンピュータではこの状況を作り出すことができ、それが「量子もつれ」というものになります！

とりあえずの量子もつれの理解としては、「現実的には作り出すことが難しい状態」で十分でしょう。

それではいよいよ、量子もつれを作ってみましょう！いわゆる「*Bell* 状態」というものを作ります。今回は、以下の式が表すものを作ってみましょう！

> $|Bell\rangle = \frac{1}{\sqrt 2}(|00\rangle + |11\rangle)$

In [ ]:
qc = QuantumCircuit(qr)

qc.h(0)
qc.cx(0, 1)

qc.draw("mpl")

量子回路ができました！しかし、これだけだと本当に量子もつれが起こっているのかわかりません。そこで、「statevector_simulator」を使って確認しましょう！

In [ ]:
backend = Aer.get_backend('statevector_simulator')
qc_transpiled = transpile(qc, backend)

job = backend.run(qc_transpiled)
state_vec = job.result().get_statevector(qc)

for i in range(4):
    print(state_vec[i])

これはまさに、

> $\frac{1}{\sqrt 2}\left(\begin{array}{c}1\\0\\0\\1\\\end{array}\right) = \frac{1}{\sqrt 2}(\left(\begin{array}{c}1\\0\\0\\0\\\end{array}\right) + \left(\begin{array}{c}0\\0\\0\\1\\\end{array}\right))$

となっていて、 *Bell* 状態の定義

> $\frac{1}{\sqrt 2}(|00\rangle + |11\rangle) := \frac{1}{\sqrt 2}(\left(\begin{array}{c}1\\0\\0\\0\\\end{array}\right) + \left(\begin{array}{c}0\\0\\0\\1\\\end{array}\right))$

と一致しています！

また、今度は「qasm_simulator」を使って、シミュレーション実験を行いましょう！これは量子回路の最後に測定を行うことでサンプリングをします。よって、先ほど作った回路を少しばかりイジる必要があります。

まず、最初に

```python
qc = QuantumCircuit(2)
```

としていましたが、もう一つ引数を付け加えます。これは古典ビットの数を入れます。

```python
qc_new = QuantumCircuit(2, 2)
```

また、これも古典レジスタを使うと以下のように書き直せます。

```python
qr = QuantumRegister(2)
cr = ClassicalRegister(2)
qc_new = QuantumCircuit(qr, cr)
```

In [ ]:
qr = QuantumRegister(2)
cr = ClassicalRegister(2)
qc_new = QuantumCircuit(qr, cr)

qc_new.h(0)
qc_new.cx(0, 1)

qc_new.draw("mpl")

先ほどと少し変わり、下に古典レジスタを表すものが出てきました！では、ここから測定に入ります。

In [ ]:
qc_new.measure(0, 0)
qc_new.measure(1, 1)

qc_new.draw("mpl")

測定に使う $measure$ というゲートの引数は (測定する量子ビットのインデックス, 測定結果を保存する古典的レジスタのインデックス) というようになります。さてこれで測定ができるので、シミュレーションをして結果を見てみましょう！

In [ ]:
backend = Aer.get_backend('qasm_simulator')
qc_transpiled = transpile(qc_new, backend)

job = backend.run(qc_transpiled, shots=1000)
counts = job.result().get_counts()
plot_histogram(counts)

このように、ほぼ同確率でこの2つの状態のみ得られました！

それでは演習をやってみましょう！

# Exercise 1

**Exercise 1-a**

今度は以下の *Bell* 状態を作ってみましょう！

> $|Bell_{another}\rangle = \frac{1}{\sqrt 2}(|01\rangle + |10\rangle)$

これを「qasm_simulator」を使って結果を出力してください。

In [ ]:
qr = QuantumRegister(2)
cr = ClassicalRegister(2)
qc = QuantumCircuit(qr, cr)

# ここから下にコードを書いてみましょう！

# ここから上にコードを書いてみましょう！

qc.draw("mpl")

In [ ]:
backend = Aer.get_backend('qasm_simulator')
qc_transpiled = transpile(qc, backend)

job = backend.run(qc_transpiled, shots=1000)
counts = job.result().get_counts()
plot_histogram(counts).savefig("myans_1a.png")

**Exercise 1-b**

余力のある人は、先ほど使った量子ゲートのみで次の $3$ 量子ビットの量子もつれを作ってみましょう！

$|ent\rangle = \frac{1}{2}(|001\rangle + |010\rangle + |100\rangle + |111\rangle)$

「qasm_simulator」を使って結果を出力してください。

In [ ]:
qr = QuantumRegister(3)
cr = ClassicalRegister(3)
qc = QuantumCircuit(qr, cr)

# ここから下にコードを書いてみましょう！

# ここから上にコードを書いてみましょう！

qc.draw("mpl")

In [ ]:
backend = Aer.get_backend('qasm_simulator')
qc_transpiled = transpile(qc, backend)

job = backend.run(qc_transpiled, shots=1000)
counts = job.result().get_counts()
plot_histogram(counts).savefig("myans_1b.png")

# 実機を用いた実験

今回の大目玉である実機での実験を行いましょう！

In [ ]:
provider = IBMProvider()
provider.backends()

使える実機の数や種類は人や時期によって変わってきます。なので毎回同じものが使えないことに注意しましょう！今回はせっかくなので、 $7$ 量子ビットのマシンを使ってみましょう。

In [ ]:
hub = "ibm-q"
group = "open"
project = "main"

backend_name = "ibm_nairobi"
backend = provider.get_backend(backend_name, instance=f"{hub}/{group}/{project}")

では、上で作った *Bell* 状態を実機で実行してみましょう！

In [ ]:
qr = QuantumRegister(2)
cr = ClassicalRegister(2)
qc = QuantumCircuit(qr, cr)

qc.h(0)
qc.cx(0, 1)

qc.measure(0, 0)
qc.measure(1, 1)

qc.draw("mpl")

改めて量子回路が用意できたので、実行してみましょう。

In [ ]:
qc_transpiled = transpile(qc, backend)
job = backend.run(qc_transpiled, shots=1000)

counts = job.result().get_counts()
plot_histogram(counts)

実機はキューに入ってから実行するまでにかなり時間がかかる場合があるので、今回は事前に実行した結果だけご紹介します！

![output.png](https://lab.quantum-computing.ibm.com/user/6270b5428acd4b124d17130d/files/others/output.png?_xsrf=2%7Ca7a4d62e%7C631b8eed51c39b4626700c02e34a7d32%7C1686117575)

少しばかりノイズが入っていることが確認できますね。量子コンピュータでは、ノイズを考慮したアルゴリズムやエラー訂正を行うための手法など様々ありますが、それはプロに聞いてみましょう(丸投げ)。

それでは演習をやってみましょう！

# Exercise 2

**Exercise 2-a**

量子ビット数が $3$ 以上のとき、有名な量子もつれとして *GHZ* 状態というものがあります。これは、

> $|GHZ\rangle = \frac{1}{\sqrt 2}(|0\rangle^{{\bigotimes} n} + |1\rangle^{{\bigotimes} n})$

で表せるものです。この演習では、 $3$ 量子ビットでの *GHZ* 状態を作り、それを実機で実行しましょう！

In [ ]:
qr = QuantumRegister(3)
cr = ClassicalRegister(3)
qc = QuantumCircuit(qr, cr)

# ここから下にコードを書いてみましょう！

# ここから上にコードを書いてみましょう！

qc.draw("mpl")

In [ ]:
hub = "ibm-q"
group = "open"
project = "main"

# ここから下にコードを書いてみましょう！
backend_name = 
# ここから上にコードを書いてみましょう！

backend = provider.get_backend(backend_name, instance=f"{hub}/{group}/{project}")

qc_transpiled = transpile(qc, backend)
job = backend.run(qc_transpiled, shots=1000)

counts = job.result().get_counts()
plot_histogram(counts).savefig("myans_2a.png")

**Exercise 2-b**

余力のある人は、今度は $7$ 量子ビットの *GHZ* 状態を作ってみましょう！また、以下のことを考えてみてください。

普通、アルゴリズムを扱う上で「計算量」というものを気にします。これは、コンピュータでの負荷を抑え、高速に計算することや、メモリを確保するためにより良いアルゴリズムを考えるというものです。量子コンピュータでのプログラミングも例外ではなく、量子回路を設計する際に、その回路の深さ( *depth* )というものをなるべく浅くすることを心がけましょう。例として $S$ ゲートというものがあり、これを2個繋げると $Z$ ゲートになるのですが、もちろん $S$ ゲートを2個続けてかけるより $Z$ ゲートを1個かけてあげる方が計算量は少なくて済みます。Qiskitでは、作った量子回路の *depth* を調べるための `QuantumCircuit.depth()` というメソッドが提供されてます。これを使って最も浅い量子回路を作成しましょう！最も浅い量子回路の *depth* は $5$ です！

In [ ]:
qr = QuantumRegister(7)
cr = ClassicalRegister(7)
qc = QuantumCircuit(qr, cr)

# ここから下にコードを書いてみましょう！

# ここから上にコードを書いてみましょう！

# 測定
for i in range(7):
    qc.measure(i, i)
    
qc.draw("mpl")

In [ ]:
if qc.depth() == 5:

    # 7量子ビットのマシンが設定されていなければ、ここから下にコードを書いてみましょう！
    
    # 7量子ビットのマシンが設定されていなければ、ここから上にコードを書いてみましょう！

    qc_transpiled = transpile(qc, backend)
    job = backend.run(qc_transpiled, shots=1000)

    counts = job.result().get_counts()
    plot_histogram(counts).savefig("myans_2b.png")
else:
    print("wrong circuit :(")